## 1. First names including all genders

In [1]:
from utils import *

exclude_u = False
ccnc = data_loader('data/data96861/ccnc.txt', first_name_only=True, exclude_u=exclude_u)
train_set, dev_set, test_set = train_dev_test_split(ccnc)
text = [t[0] for t in train_set]
V = TextVectorizer(list)
V.build_vocab(text)
print("Number of vocab (char):", len(V))

Number of vocab (char): 6254


In [2]:
import paddle
from models import BoW, LogisticRegression, CNN, LSTM


def get_model(model):
    model = paddle.Model(model)
    optimizer = paddle.optimizer.Adam(
    parameters=model.parameters(), learning_rate=5e-4)
    criterion = paddle.nn.CrossEntropyLoss()
    metric = paddle.metric.Accuracy()
    model.prepare(optimizer, criterion, metric)
    return model


def do_train_and_evaluate(network, 
                          epochs=10, 
                          batch_size=1024,
                          log_freq=1000, 
                          exclude_u=True):
    
    include_seq_len = False
    num_class = 2 if exclude_u else 3
    if network.lower() == "bow":
        model = BoW(len(V), num_class)
    elif network.lower() == 'cnn':
        model = CNN(len(V), num_class)
    elif network.lower() == 'lstm':
        model = LSTM(len(V), num_class)
        include_seq_len = True
    elif network.lower() == 'lr':
        model = LogisticRegression(len(V), num_class)
    else:
        raise ValueError(f"Only supports: bow, cnn, lstm, gru, not {network}")
    
    trans_fn = get_trans_fn(V, include_seq_len=include_seq_len)
    batchify_fn = get_batchify_fn(include_seq_len=include_seq_len)
    train_loader = create_dataloader(train_set, trans_fn, batchify_fn, batch_size=batch_size)
    dev_loader = create_dataloader(dev_set, trans_fn, batchify_fn, batch_size=batch_size)
    test_loader = create_dataloader(test_set, trans_fn, batchify_fn, shuffle=False, batch_size=batch_size)
    
    model = get_model(model)
    model.fit(train_loader, dev_loader, epochs=epochs, verbose=2, log_freq=log_freq)
    model.evaluate(test_loader, verbose=2, log_freq=log_freq)

In [3]:
do_train_and_evaluate('bow', 10, exclude_u=exclude_u)

W0402 02:24:44.953387   940 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 10.1, Runtime API Version: 10.1
W0402 02:24:44.958230   940 device_context.cc:465] device: 0, cuDNN Version: 7.6.


The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/10
step 1000/2144 - loss: 0.1225 - acc: 0.9373 - 11ms/step
step 2000/2144 - loss: 0.1266 - acc: 0.9449 - 11ms/step
step 2144/2144 - loss: 0.1013 - acc: 0.9455 - 11ms/step
Eval begin...
step 715/715 - loss: 0.1245 - acc: 0.9542 - 9ms/step
Eval samples: 731622
Epoch 2/10
step 1000/2144 - loss: 0.1110 - acc: 0.9547 - 11ms/step
step 2000/2144 - loss: 0.1337 - acc: 0.9554 - 11ms/step
step 2144/2144 - loss: 0.1419 - acc: 0.9555 - 11ms/step
Eval begin...
step 715/715 - loss: 0.1404 - acc: 0.9568 - 9ms/step
Eval samples: 731622
Epoch 3/10
step 1000/2144 - loss: 0.0967 - acc: 0.9574 - 11ms/step
step 2000/2144 - loss: 0.1162 - acc: 0.9577 - 10ms/step
step 2144/2144 - loss: 0.0963 - acc: 0.9577 - 10ms/step
Eval begin...
step 715/715 - loss: 0.0995 - acc: 0.9579 - 12ms/step
Eval samples: 731622
Epoch 4/10
step 1000/2144 - loss: 0.0950 - acc: 0.9590 - 11ms/step
step 2000/2144 - loss

In [4]:
do_train_and_evaluate('cnn', 10, exclude_u=exclude_u)

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/10
step 1000/2144 - loss: 0.1270 - acc: 0.9406 - 11ms/step
step 2000/2144 - loss: 0.1268 - acc: 0.9474 - 11ms/step
step 2144/2144 - loss: 0.0922 - acc: 0.9479 - 11ms/step
Eval begin...
step 715/715 - loss: 0.0814 - acc: 0.9565 - 10ms/step
Eval samples: 731622
Epoch 2/10
step 1000/2144 - loss: 0.1204 - acc: 0.9572 - 13ms/step
step 2000/2144 - loss: 0.0996 - acc: 0.9578 - 12ms/step
step 2144/2144 - loss: 0.0927 - acc: 0.9578 - 12ms/step
Eval begin...
step 715/715 - loss: 0.0878 - acc: 0.9588 - 10ms/step
Eval samples: 731622
Epoch 3/10
step 1000/2144 - loss: 0.1042 - acc: 0.9602 - 11ms/step
step 2000/2144 - loss: 0.0771 - acc: 0.9605 - 11ms/step
step 2144/2144 - loss: 0.0675 - acc: 0.9605 - 11ms/step
Eval begin...
step 715/715 - loss: 0.0713 - acc: 0.9608 - 10ms/step
Eval samples: 731622
Epoch 4/10
step 1000/2144 - loss: 0.0785 - acc: 0.9618 - 12ms/step
step 2000/2144 - lo

In [5]:
do_train_and_evaluate('lr', 10, exclude_u=exclude_u)

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/10
step 1000/2144 - loss: 0.1472 - acc: 0.9348 - 10ms/step
step 2000/2144 - loss: 0.1274 - acc: 0.9429 - 10ms/step
step 2144/2144 - loss: 0.1387 - acc: 0.9435 - 10ms/step
Eval begin...
step 715/715 - loss: 0.1382 - acc: 0.9525 - 9ms/step
Eval samples: 731622
Epoch 2/10
step 1000/2144 - loss: 0.1353 - acc: 0.9531 - 10ms/step
step 2000/2144 - loss: 0.1591 - acc: 0.9533 - 10ms/step
step 2144/2144 - loss: 0.1266 - acc: 0.9533 - 10ms/step
Eval begin...
step 715/715 - loss: 0.1464 - acc: 0.9538 - 9ms/step
Eval samples: 731622
Epoch 3/10
step 1000/2144 - loss: 0.1057 - acc: 0.9541 - 11ms/step
step 2000/2144 - loss: 0.1338 - acc: 0.9542 - 11ms/step
step 2144/2144 - loss: 0.1217 - acc: 0.9542 - 10ms/step
Eval begin...
step 715/715 - loss: 0.1068 - acc: 0.9541 - 9ms/step
Eval samples: 731622
Epoch 4/10
step 1000/2144 - loss: 0.1429 - acc: 0.9546 - 10ms/step
step 2000/2144 - loss:

In [6]:
do_train_and_evaluate('lstm', 10, exclude_u=exclude_u)

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/10
step 1000/2144 - loss: 0.1293 - acc: 0.9337 - 15ms/step
step 2000/2144 - loss: 0.1038 - acc: 0.9442 - 14ms/step
step 2144/2144 - loss: 0.1615 - acc: 0.9449 - 14ms/step
Eval begin...
step 715/715 - loss: 0.1316 - acc: 0.9566 - 12ms/step
Eval samples: 731622
Epoch 2/10
step 1000/2144 - loss: 0.0971 - acc: 0.9568 - 14ms/step
step 2000/2144 - loss: 0.0764 - acc: 0.9572 - 14ms/step
step 2144/2144 - loss: 0.0980 - acc: 0.9572 - 14ms/step
Eval begin...
step 715/715 - loss: 0.0940 - acc: 0.9584 - 11ms/step
Eval samples: 731622
Epoch 3/10
step 1000/2144 - loss: 0.0919 - acc: 0.9591 - 14ms/step
step 2000/2144 - loss: 0.0890 - acc: 0.9594 - 14ms/step
step 2144/2144 - loss: 0.0798 - acc: 0.9594 - 13ms/step
Eval begin...
step 715/715 - loss: 0.0905 - acc: 0.9599 - 11ms/step
Eval samples: 731622
Epoch 4/10
step 1000/2144 - loss: 0.0845 - acc: 0.9609 - 15ms/step
step 2000/2144 - lo

## 2. First names excluding unknown gender

In [7]:
exclude_u = True
ccnc = data_loader('data/data96861/ccnc.txt', first_name_only=True, exclude_u=exclude_u)
train_set, dev_set, test_set = train_dev_test_split(ccnc)
text = [t[0] for t in train_set]
V = TextVectorizer(list)
V.build_vocab(text)
print("Number of vocab (char):", len(V))

Number of vocab (char): 6249


In [8]:
do_train_and_evaluate('bow', 10, exclude_u=exclude_u)

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/10
step 1000/2089 - loss: 0.0531 - acc: 0.9658 - 11ms/step
step 2000/2089 - loss: 0.0439 - acc: 0.9721 - 11ms/step
step 2089/2089 - loss: 0.0532 - acc: 0.9724 - 11ms/step
Eval begin...
step 697/697 - loss: 0.0352 - acc: 0.9795 - 10ms/step
Eval samples: 712757
Epoch 2/10
step 1000/2089 - loss: 0.0616 - acc: 0.9804 - 11ms/step
step 2000/2089 - loss: 0.0422 - acc: 0.9806 - 11ms/step
step 2089/2089 - loss: 0.0532 - acc: 0.9806 - 11ms/step
Eval begin...
step 697/697 - loss: 0.0613 - acc: 0.9818 - 10ms/step
Eval samples: 712757
Epoch 3/10
step 1000/2089 - loss: 0.0398 - acc: 0.9822 - 11ms/step
step 2000/2089 - loss: 0.0427 - acc: 0.9823 - 11ms/step
step 2089/2089 - loss: 0.0404 - acc: 0.9823 - 11ms/step
Eval begin...
step 697/697 - loss: 0.0113 - acc: 0.9824 - 10ms/step
Eval samples: 712757
Epoch 4/10
step 1000/2089 - loss: 0.0392 - acc: 0.9829 - 11ms/step
step 2000/2089 - lo

In [9]:
do_train_and_evaluate('cnn', 10, exclude_u=exclude_u)

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/10
step 1000/2089 - loss: 0.0624 - acc: 0.9617 - 12ms/step
step 2000/2089 - loss: 0.0622 - acc: 0.9704 - 12ms/step
step 2089/2089 - loss: 0.0289 - acc: 0.9709 - 12ms/step
Eval begin...
step 697/697 - loss: 0.0712 - acc: 0.9811 - 10ms/step
Eval samples: 712757
Epoch 2/10
step 1000/2089 - loss: 0.0214 - acc: 0.9828 - 12ms/step
step 2000/2089 - loss: 0.0426 - acc: 0.9834 - 13ms/step
step 2089/2089 - loss: 0.0476 - acc: 0.9835 - 13ms/step
Eval begin...
step 697/697 - loss: 0.0260 - acc: 0.9845 - 10ms/step
Eval samples: 712757
Epoch 3/10
step 1000/2089 - loss: 0.0332 - acc: 0.9856 - 12ms/step
step 2000/2089 - loss: 0.0209 - acc: 0.9859 - 12ms/step
step 2089/2089 - loss: 0.0323 - acc: 0.9859 - 12ms/step
Eval begin...
step 697/697 - loss: 0.0440 - acc: 0.9848 - 10ms/step
Eval samples: 712757
Epoch 4/10
step 1000/2089 - loss: 0.0228 - acc: 0.9868 - 12ms/step
step 2000/2089 - lo

In [10]:
do_train_and_evaluate('lr', 10, exclude_u=exclude_u)

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/10
step 1000/2089 - loss: 0.0698 - acc: 0.9607 - 10ms/step
step 2000/2089 - loss: 0.0596 - acc: 0.9688 - 10ms/step
step 2089/2089 - loss: 0.0505 - acc: 0.9692 - 10ms/step
Eval begin...
step 697/697 - loss: 0.1110 - acc: 0.9784 - 10ms/step
Eval samples: 712757
Epoch 2/10
step 1000/2089 - loss: 0.0792 - acc: 0.9789 - 10ms/step
step 2000/2089 - loss: 0.0532 - acc: 0.9791 - 10ms/step
step 2089/2089 - loss: 0.0520 - acc: 0.9792 - 10ms/step
Eval begin...
step 697/697 - loss: 0.0482 - acc: 0.9801 - 10ms/step
Eval samples: 712757
Epoch 3/10
step 1000/2089 - loss: 0.0498 - acc: 0.9800 - 10ms/step
step 2000/2089 - loss: 0.0338 - acc: 0.9802 - 11ms/step
step 2089/2089 - loss: 0.0551 - acc: 0.9802 - 11ms/step
Eval begin...
step 697/697 - loss: 0.0078 - acc: 0.9801 - 9ms/step
Eval samples: 712757
Epoch 4/10
step 1000/2089 - loss: 0.0510 - acc: 0.9805 - 11ms/step
step 2000/2089 - los

In [11]:
do_train_and_evaluate('lstm', 10, exclude_u=exclude_u)

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/10
step 1000/2089 - loss: 0.0639 - acc: 0.9567 - 14ms/step
step 2000/2089 - loss: 0.0329 - acc: 0.9678 - 14ms/step
step 2089/2089 - loss: 0.0339 - acc: 0.9682 - 14ms/step
Eval begin...
step 697/697 - loss: 0.0138 - acc: 0.9797 - 14ms/step
Eval samples: 712757
Epoch 2/10
step 1000/2089 - loss: 0.0503 - acc: 0.9806 - 14ms/step
step 2000/2089 - loss: 0.0418 - acc: 0.9811 - 14ms/step
step 2089/2089 - loss: 0.0350 - acc: 0.9811 - 14ms/step
Eval begin...
step 697/697 - loss: 0.1620 - acc: 0.9820 - 12ms/step
Eval samples: 712757
Epoch 3/10
step 1000/2089 - loss: 0.0395 - acc: 0.9824 - 15ms/step
step 2000/2089 - loss: 0.0526 - acc: 0.9827 - 14ms/step
step 2089/2089 - loss: 0.0652 - acc: 0.9827 - 14ms/step
Eval begin...
step 697/697 - loss: 0.0180 - acc: 0.9829 - 12ms/step
Eval samples: 712757
Epoch 4/10
step 1000/2089 - loss: 0.0286 - acc: 0.9834 - 14ms/step
step 2000/2089 - lo

## 3. Full names inlcuding all genders

In [12]:
exclude_u = False
ccnc = data_loader('data/data96861/ccnc.txt', first_name_only=False, exclude_u=exclude_u)
train_set, dev_set, test_set = train_dev_test_split(ccnc)
text = [t[0] for t in train_set]
V = TextVectorizer(list)
V.build_vocab(text)
print("Number of vocab (char):", len(V))

Number of vocab (char): 6255


In [13]:
do_train_and_evaluate('bow', 10, exclude_u=exclude_u)

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/10
step 1000/2144 - loss: 0.1605 - acc: 0.9190 - 14ms/step
step 2000/2144 - loss: 0.1529 - acc: 0.9272 - 14ms/step
step 2144/2144 - loss: 0.1300 - acc: 0.9279 - 14ms/step
Eval begin...
step 715/715 - loss: 0.1401 - acc: 0.9380 - 12ms/step
Eval samples: 731622
Epoch 2/10
step 1000/2144 - loss: 0.1330 - acc: 0.9395 - 16ms/step
step 2000/2144 - loss: 0.1575 - acc: 0.9412 - 15ms/step
step 2144/2144 - loss: 0.1319 - acc: 0.9414 - 15ms/step
Eval begin...
step 715/715 - loss: 0.1267 - acc: 0.9452 - 13ms/step
Eval samples: 731622
Epoch 3/10
step 1000/2144 - loss: 0.1118 - acc: 0.9469 - 14ms/step
step 2000/2144 - loss: 0.1399 - acc: 0.9476 - 14ms/step
step 2144/2144 - loss: 0.1129 - acc: 0.9477 - 14ms/step
Eval begin...
step 715/715 - loss: 0.1210 - acc: 0.9494 - 13ms/step
Eval samples: 731622
Epoch 4/10
step 1000/2144 - loss: 0.1157 - acc: 0.9508 - 14ms/step
step 2000/2144 - lo

In [14]:
do_train_and_evaluate('cnn', 10, exclude_u=exclude_u)

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/10
step 1000/2144 - loss: 0.1614 - acc: 0.9173 - 15ms/step
step 2000/2144 - loss: 0.1653 - acc: 0.9265 - 15ms/step
step 2144/2144 - loss: 0.2029 - acc: 0.9272 - 15ms/step
Eval begin...
step 715/715 - loss: 0.1513 - acc: 0.9392 - 13ms/step
Eval samples: 731622
Epoch 2/10
step 1000/2144 - loss: 0.1536 - acc: 0.9427 - 14ms/step
step 2000/2144 - loss: 0.1372 - acc: 0.9454 - 14ms/step
step 2144/2144 - loss: 0.1378 - acc: 0.9457 - 14ms/step
Eval begin...
step 715/715 - loss: 0.1497 - acc: 0.9503 - 13ms/step
Eval samples: 731622
Epoch 3/10
step 1000/2144 - loss: 0.1133 - acc: 0.9530 - 16ms/step
step 2000/2144 - loss: 0.0957 - acc: 0.9534 - 15ms/step
step 2144/2144 - loss: 0.1092 - acc: 0.9535 - 15ms/step
Eval begin...
step 715/715 - loss: 0.1125 - acc: 0.9539 - 13ms/step
Eval samples: 731622
Epoch 4/10
step 1000/2144 - loss: 0.1097 - acc: 0.9569 - 15ms/step
step 2000/2144 - lo

In [15]:
do_train_and_evaluate('lr', 10, exclude_u=exclude_u)

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/10
step 1000/2144 - loss: 0.1883 - acc: 0.9156 - 13ms/step
step 2000/2144 - loss: 0.1512 - acc: 0.9245 - 14ms/step
step 2144/2144 - loss: 0.1241 - acc: 0.9251 - 14ms/step
Eval begin...
step 715/715 - loss: 0.2307 - acc: 0.9346 - 13ms/step
Eval samples: 731622
Epoch 2/10
step 1000/2144 - loss: 0.1843 - acc: 0.9343 - 14ms/step
step 2000/2144 - loss: 0.1644 - acc: 0.9346 - 13ms/step
step 2144/2144 - loss: 0.1548 - acc: 0.9347 - 13ms/step
Eval begin...
step 715/715 - loss: 0.1462 - acc: 0.9350 - 13ms/step
Eval samples: 731622
Epoch 3/10
step 1000/2144 - loss: 0.1595 - acc: 0.9357 - 14ms/step
step 2000/2144 - loss: 0.1679 - acc: 0.9353 - 13ms/step
step 2144/2144 - loss: 0.1608 - acc: 0.9352 - 13ms/step
Eval begin...
step 715/715 - loss: 0.1757 - acc: 0.9354 - 13ms/step
Eval samples: 731622
Epoch 4/10
step 1000/2144 - loss: 0.1743 - acc: 0.9354 - 15ms/step
step 2000/2144 - lo

In [16]:
do_train_and_evaluate('lstm', 10, exclude_u=exclude_u)

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/10
step 1000/2144 - loss: 0.1124 - acc: 0.9255 - 16ms/step
step 2000/2144 - loss: 0.0910 - acc: 0.9405 - 16ms/step
step 2144/2144 - loss: 0.1221 - acc: 0.9417 - 16ms/step
Eval begin...
step 715/715 - loss: 0.1285 - acc: 0.9585 - 15ms/step
Eval samples: 731622
Epoch 2/10
step 1000/2144 - loss: 0.0777 - acc: 0.9605 - 16ms/step
step 2000/2144 - loss: 0.0951 - acc: 0.9623 - 17ms/step
step 2144/2144 - loss: 0.0710 - acc: 0.9626 - 17ms/step
Eval begin...
step 715/715 - loss: 0.0824 - acc: 0.9652 - 14ms/step
Eval samples: 731622
Epoch 3/10
step 1000/2144 - loss: 0.0829 - acc: 0.9674 - 17ms/step
step 2000/2144 - loss: 0.0642 - acc: 0.9680 - 17ms/step
step 2144/2144 - loss: 0.0681 - acc: 0.9681 - 17ms/step
Eval begin...
step 715/715 - loss: 0.0880 - acc: 0.9688 - 15ms/step
Eval samples: 731622
Epoch 4/10
step 1000/2144 - loss: 0.0766 - acc: 0.9702 - 16ms/step
step 2000/2144 - lo

## 4. Full names excluding unknown gender

In [17]:
exclude_u = True
ccnc = data_loader('data/data96861/ccnc.txt', first_name_only=False, exclude_u=exclude_u)
train_set, dev_set, test_set = train_dev_test_split(ccnc)
text = [t[0] for t in train_set]
V = TextVectorizer(list)
V.build_vocab(text)
print("Number of vocab (char):", len(V))

Number of vocab (char): 6251


In [18]:
do_train_and_evaluate('bow', 10, exclude_u=exclude_u)

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/10
step 1000/2089 - loss: 0.1061 - acc: 0.9451 - 14ms/step
step 2000/2089 - loss: 0.1193 - acc: 0.9519 - 14ms/step
step 2089/2089 - loss: 0.1205 - acc: 0.9523 - 14ms/step
Eval begin...
step 697/697 - loss: 0.0319 - acc: 0.9603 - 12ms/step
Eval samples: 712757
Epoch 2/10
step 1000/2089 - loss: 0.1014 - acc: 0.9616 - 14ms/step
step 2000/2089 - loss: 0.0699 - acc: 0.9633 - 14ms/step
step 2089/2089 - loss: 0.0492 - acc: 0.9634 - 14ms/step
Eval begin...
step 697/697 - loss: 0.0874 - acc: 0.9675 - 13ms/step
Eval samples: 712757
Epoch 3/10
step 1000/2089 - loss: 0.0815 - acc: 0.9695 - 14ms/step
step 2000/2089 - loss: 0.0502 - acc: 0.9709 - 14ms/step
step 2089/2089 - loss: 0.0909 - acc: 0.9710 - 14ms/step
Eval begin...
step 697/697 - loss: 0.0598 - acc: 0.9733 - 12ms/step
Eval samples: 712757
Epoch 4/10
step 1000/2089 - loss: 0.0542 - acc: 0.9746 - 16ms/step
step 2000/2089 - lo

In [19]:
do_train_and_evaluate('cnn', 10, exclude_u=exclude_u)

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/10
step 1000/2089 - loss: 0.0855 - acc: 0.9434 - 15ms/step
step 2000/2089 - loss: 0.0787 - acc: 0.9522 - 15ms/step
step 2089/2089 - loss: 0.0474 - acc: 0.9528 - 15ms/step
Eval begin...
step 697/697 - loss: 0.0450 - acc: 0.9659 - 13ms/step
Eval samples: 712757
Epoch 2/10
step 1000/2089 - loss: 0.0797 - acc: 0.9695 - 15ms/step
step 2000/2089 - loss: 0.0767 - acc: 0.9715 - 16ms/step
step 2089/2089 - loss: 0.0928 - acc: 0.9716 - 16ms/step
Eval begin...
step 697/697 - loss: 0.0702 - acc: 0.9746 - 13ms/step
Eval samples: 712757
Epoch 3/10
step 1000/2089 - loss: 0.0719 - acc: 0.9768 - 15ms/step
step 2000/2089 - loss: 0.0494 - acc: 0.9770 - 15ms/step
step 2089/2089 - loss: 0.0332 - acc: 0.9770 - 15ms/step
Eval begin...
step 697/697 - loss: 0.0147 - acc: 0.9769 - 13ms/step
Eval samples: 712757
Epoch 4/10
step 1000/2089 - loss: 0.0413 - acc: 0.9793 - 15ms/step
step 2000/2089 - lo

In [20]:
do_train_and_evaluate('lr', 10, exclude_u=exclude_u)

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/10
step 1000/2089 - loss: 0.1208 - acc: 0.9396 - 14ms/step
step 2000/2089 - loss: 0.0876 - acc: 0.9485 - 14ms/step
step 2089/2089 - loss: 0.1086 - acc: 0.9490 - 14ms/step
Eval begin...
step 697/697 - loss: 0.0615 - acc: 0.9590 - 13ms/step
Eval samples: 712757
Epoch 2/10
step 1000/2089 - loss: 0.0964 - acc: 0.9588 - 14ms/step
step 2000/2089 - loss: 0.0992 - acc: 0.9591 - 14ms/step
step 2089/2089 - loss: 0.1217 - acc: 0.9591 - 14ms/step
Eval begin...
step 697/697 - loss: 0.1792 - acc: 0.9596 - 13ms/step
Eval samples: 712757
Epoch 3/10
step 1000/2089 - loss: 0.1123 - acc: 0.9598 - 14ms/step
step 2000/2089 - loss: 0.0873 - acc: 0.9596 - 14ms/step
step 2089/2089 - loss: 0.1050 - acc: 0.9596 - 14ms/step
Eval begin...
step 697/697 - loss: 0.0591 - acc: 0.9598 - 13ms/step
Eval samples: 712757
Epoch 4/10
step 1000/2089 - loss: 0.1029 - acc: 0.9599 - 14ms/step
step 2000/2089 - lo

In [21]:
do_train_and_evaluate('lstm', 10, exclude_u=exclude_u)

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/10
step 1000/2089 - loss: 0.0400 - acc: 0.9510 - 17ms/step
step 2000/2089 - loss: 0.0470 - acc: 0.9649 - 19ms/step
step 2089/2089 - loss: 0.0586 - acc: 0.9655 - 19ms/step
Eval begin...
step 697/697 - loss: 0.0381 - acc: 0.9803 - 17ms/step
Eval samples: 712757
Epoch 2/10
step 1000/2089 - loss: 0.0513 - acc: 0.9818 - 16ms/step
step 2000/2089 - loss: 0.0425 - acc: 0.9826 - 16ms/step
step 2089/2089 - loss: 0.0506 - acc: 0.9827 - 16ms/step
Eval begin...
step 697/697 - loss: 0.0517 - acc: 0.9844 - 15ms/step
Eval samples: 712757
Epoch 3/10
step 1000/2089 - loss: 0.0302 - acc: 0.9850 - 18ms/step
step 2000/2089 - loss: 0.0346 - acc: 0.9853 - 17ms/step
step 2089/2089 - loss: 0.0380 - acc: 0.9853 - 17ms/step
Eval begin...
step 697/697 - loss: 0.1198 - acc: 0.9857 - 15ms/step
Eval samples: 712757
Epoch 4/10
step 1000/2089 - loss: 0.0343 - acc: 0.9862 - 19ms/step
step 2000/2089 - lo